In [316]:
import pandas as pd

import random
import json

import os

In [317]:
# train index
# row -> [(item_id, time_ms, click, like, follow, comment, forward, longview, pltr, pwtr, pcmtr, pftr, plvtr), ()]
def index_encoding_item(ltr_sample):
    item_id_set = set()
    for sample in ltr_sample:
        for item in sample:
            item_id_set.add(item[0])
    print ("item number=" ,len(item_id_set))
    
    item_num2id = list(item_id_set)
    item_num2id.sort()

    item_id2num = {}
    for num in range(0, len(item_id_set)):
        item_id2num[item_num2id[num]] = num

    ltr_sample_number = []
    for sample in ltr_sample:
        ltr_sample_single_item = []
        for (item_id, time_ms, click, like, follow, comment, forward, longview, pltr, pwtr, pcmtr, pftr, plvtr) in sample:
            ltr_sample_single_item.append([item_id2num[item_id], time_ms, click, like, follow, comment, forward, longview, pltr, pwtr, pcmtr, pftr, plvtr])
        ltr_sample_number.append(ltr_sample_single_item)

    ltr_sample = ltr_sample_number
    return ltr_sample, item_id2num

In [318]:
# filter test 
def filter_test_data(ltr_sample, item_id2num):
    new_data = []
    for sample in ltr_sample:
        new_data_single_item = []
        for (item_id, time_ms, click, like, follow, comment, forward, longview, pltr, pwtr, pcmtr, pftr, plvtr) in sample:
            if item_id in item_id2num:
                new_data_single_item.append([item_id2num[item_id], time_ms, click, like, follow, comment, forward, longview, pltr, pwtr, pcmtr, pftr, plvtr])
        new_data.append(new_data_single_item)

    return new_data

# write json
def write_data(path, data):
    f = open(path, 'w')
    jsObj = json.dumps(data)
    f.write(jsObj)
    f.close()

In [333]:
DIR = '../CLM/clm_pretraining_with_kuairand/dataset/KuaiRand/'
origin_ltr_data_path = DIR + 'kuairand_ltr_data.json'

path_ltr_train = DIR + 'kuairand_ltr_data_train.json'
path_ltr_test = DIR + 'kuairand_ltr_data_test.json'

# 1 load data
with open(origin_ltr_data_path) as f:
    line = f.readline()
    data = json.loads(line)
f.close()
row_num = len(data)
print ("sample number=", row_num, ", data_path=", origin_ltr_data_path)
print("data[:2]=", data[:2])

sample number= 843819 , data_path= ../CLM/clm_pretraining_with_kuairand/dataset/KuaiRand/kuairand_ltr_data.json
data[:2]= [[0, 25588, 1649467982289, 0, 0, 0, 0, 0, 0, 0.0545779, 0.03012991, 0.01782736, 0.0091756, 0.6946429], [0, 7695, 1649477382190, 0, 0, 0, 0, 0, 0, 0.06123724, 0.02370021, 0.01699675, 0.01140463, 0.70763409]]


In [334]:
# 1 N user && N user_history_list

user_set = set()
for (user_id, item_id, time_ms, click, like, follow, comment, forward, longview, pltr, pwtr, pcmtr, pftr, plvtr) in data:
    user_set.add(user_id)
user_num = len(user_set)
print("user_num=", user_num) 

user_num= 960


In [335]:
user_history_list = []
for i in range(user_num):
    user_history_list.append([])
for (user_id, item_id, time_ms, click, like, follow, comment, forward, longview, pltr, pwtr, pcmtr, pftr, plvtr) in data:
    user_history_list[user_id].append([item_id, time_ms, click, like, follow, comment, forward, longview, pltr, pwtr, pcmtr, pftr, plvtr])
# user_history_list[0]

In [336]:
print("len(user_history_list)=", len(user_history_list))

len(user_history_list)= 960


In [337]:
# 2 user_history_list sort by time_ms
# row -> [（item_id, time_ms, click, like, follow, comment, forward, longview, pltr, pwtr, pcmtr, pftr, plvtr）, ()]
user_history_list_by_time = []
for row in user_history_list:
    sorted_row = sorted(row, key=lambda x: x[1])
    user_history_list_by_time.append(row)

In [338]:
# 3  count distribution
dic_action_list_length = {}
for row in user_history_list_by_time:
    length = len(row)
    try: dic_action_list_length[length] += 1
    except: dic_action_list_length[length] = 1

sorted_keys = sorted(dic_action_list_length.keys())

sorted_dict = {}
for key in sorted_keys:
    sorted_dict[key] = dic_action_list_length[key]

for key, value in sorted_dict.items():
    print ("length=", key, " num=", value)
    break
print ("len(sorted_dict)=", len(sorted_dict))

length= 12  num= 1
len(sorted_dict)= 746


In [339]:
dic_table = {}
for key, value in sorted_dict.items():
    if (key <= 100):
        try:dic_table['0-100'] += value;
        except: dic_table['0-100'] = value
    elif (key <= 200):
        try:dic_table['100-200'] += value;
        except: dic_table['100-200'] = value
    elif (key <= 300):
        try:dic_table['200-300'] += value;
        except: dic_table['200-300'] = value
    elif (key <= 400):
        try:dic_table['300-400'] += value;
        except: dic_table['300-400'] = value
    elif (key <= 500):
        try:dic_table['400-500'] += value;
        except: dic_table['400-500'] = value
    elif (key <= 1000):
        try:dic_table['500-1000'] += value;
        except: dic_table['500-1000'] = value
    elif (key <= 2000):
        try:dic_table['1000-2000'] += value;
        except: dic_table['1000-2000'] = value
    elif (key <= 2500):
        try:dic_table['2000-2500'] += value;
        except: dic_table['2000-2500'] = value
    elif (key <= 6000):
        try:dic_table['2500-6000'] += value;
        except: dic_table['2500-6000'] = value

for key, value in dic_table.items():
    print("ation_list length:", key, "num:", value)

ation_list length: 0-100 num: 65
ation_list length: 100-200 num: 80
ation_list length: 200-300 num: 94
ation_list length: 300-400 num: 71
ation_list length: 400-500 num: 69
ation_list length: 500-1000 num: 267
ation_list length: 1000-2000 num: 229
ation_list length: 2000-2500 num: 40
ation_list length: 2500-6000 num: 45


In [340]:
# 4 set & step=
# row -> [（item_id, time_ms, click, like, follow, comment, forward, longview, pltr, pwtr, pcmtr, pftr, plvtr）, ()]
ltr_sample = []
candidate_set_size = 100
step = 30
for row in user_history_list_by_time:
    length = len(row)

    if length <= candidate_set_size:
        ltr_sample.append(row)
    else:
        start = 0
        end = candidate_set_size
        while end <= length:
            ltr_sample.append(row[start:end])
            start = start+step
            end = start+candidate_set_size
print("len(user_history_list_by_time)=", len(user_history_list_by_time))
print("step=", step, ", candidate_set_size=", candidate_set_size ,", len(ltr_sample)=", len(ltr_sample))
        

len(user_history_list_by_time)= 960
step= 30 , candidate_set_size= 100 , len(ltr_sample)= 25541


In [341]:
# 5 shuffle
random.shuffle(ltr_sample)

In [342]:
# 6 spilte train:test=9:1
sample_number = len(ltr_sample)
train_number = int(len(ltr_sample) / 10) * 9
test_number = sample_number - train_number
print ("sample_number=",sample_number, ", train_number=", train_number, ", test_number=", test_number)

train_data_ltr_sample = ltr_sample[0:train_number]
test_data_ltr_sample = ltr_sample[train_number:]

print ("len(train_data_ltr_sample)=", len(train_data_ltr_sample), ", len(test_data_ltr_sample)=", len(test_data_ltr_sample))

sample_number= 25541 , train_number= 22986 , test_number= 2555
len(train_data_ltr_sample)= 22986 , len(test_data_ltr_sample)= 2555


In [343]:
# train_data_ltr_sample[0]

In [344]:
# 7 train index
train_data_ltr_sample, item_id2num = index_encoding_item(train_data_ltr_sample)

item number= 25701


In [345]:
# 8 test item filter
test_data_ltr_sample = filter_test_data(test_data_ltr_sample, item_id2num)
print ("test sample number=", len(test_data_ltr_sample))
# test_data_ltr_sample[0]

test sample number= 2555


In [346]:
write_data(path_ltr_train, train_data_ltr_sample)
write_data(path_ltr_test, test_data_ltr_sample)